## Vector Indexing Tutorial

### BM25

In [1]:
import torch
from torch import nn
from torch.functional import tensordot
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.nn import CosineEmbeddingLoss
from torch import Tensor

from transformers import AutoModel
from transformers import AutoTokenizer
from transformers import DPRContextEncoder
from typing import List, Dict

import os
from tqdm import tqdm
import numpy as np
import json
import pandas as pd

/home/kirekara/anaconda3/lib/python3.11/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


==============================WARNING: DEPRECATED!==============================
WARNING! This version of bitsandbytes is deprecated. Please switch to `pip install bitsandbytes` and the new repo: https://github.com/TimDettmers/bitsandbytes
==============================WARNING: DEPRECATED!==============================


2024-07-07 13:26:47.437247: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-07 13:26:47.470542: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-07 13:26:47.470570: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-07 13:26:47.471393: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-07 13:26:47.477044: I tensorflow/core/platform/cpu_feature_guar

### Neural Retrieval

Lets make it a huggingface dataset out of pure convenience

In [2]:
from datasets import Dataset

imdb_dataset = Dataset.load_from_disk("imdb_top_10k_embeddings_dataset")
imdb_embeddings = np.load("imdb_top_10k_embeddings.npy")

In [3]:
imdb_dataset

Dataset({
    features: ['ID', 'Movie Name', 'Rating', 'Runtime', 'Genre', 'Metascore', 'Plot', 'Directors', 'Stars', 'Votes', 'Gross', 'Link', 'text', 'embeddings'],
    num_rows: 9999
})

In [4]:
imdb_embeddings

array([[ 0.02105394,  0.06378748, -0.04121039, ...,  0.00074133,
         0.00416954,  0.00915009],
       [-0.0132991 ,  0.03195634, -0.04103773, ..., -0.02429469,
         0.02605752,  0.00711353],
       [ 0.02886778, -0.04718696, -0.02782897, ..., -0.01550656,
        -0.01053093, -0.02435608],
       ...,
       [ 0.03597383, -0.04812222, -0.04116966, ...,  0.05863594,
        -0.02354652,  0.01549213],
       [ 0.03175337,  0.0090703 , -0.04926109, ...,  0.01373126,
        -0.03838968, -0.05070541],
       [ 0.01239666,  0.00556153, -0.02697754, ..., -0.01962082,
        -0.02398801,  0.01530278]], dtype=float32)

### Dense Embedding

In [5]:
class Transformer_embedder(nn.Module):
    def __init__(self, feat_extractor_name: str = ''):
        """Transformer Embedding model

        Args:
            feat_extractor_name (str, optional): Name of the feature extracator from HF hub or torch Hub.
        """        
        super(Transformer_embedder, self).__init__()
        

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.feat_extractor_name = feat_extractor_name

        if 'dpr' in feat_extractor_name.lower():
            feat_extractor = DPRContextEncoder.from_pretrained(feat_extractor_name)
        else:
            feat_extractor = AutoModel.from_pretrained(feat_extractor_name)
            
        self.tokenizer = AutoTokenizer.from_pretrained(feat_extractor_name)

        
        self.normalize = True
        self.feat_extractor = feat_extractor
        self.embeding_shape = self.get_extractor_output_shape() 
                            

    def get_extractor_output_shape(self):
        last_layer = list(self.feat_extractor.named_children())[-1]

        if hasattr( list(last_layer[1].modules())[1] , 'out_features'):
            shape = list(last_layer[1].modules())[1].out_features
        else:
            shape = self.feat_extractor.config.hidden_size

        return shape
    
    def mean_pooling(self, model_output:Tensor, attention_mask:Tensor):
        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


    def pool(self, embedding:Tensor, attention_mask:Tensor, pool_type:str = 'mean'):
        
        if 'mean' in pool_type:
            pooled = self.mean_pooling(embedding, attention_mask)
        else:
            pooled = embedding.last_hidden_state[:, 0, :]

        return pooled

    def __call__(self, input_ids:Tensor, attention_mask:Tensor, labels: Tensor = None, **kwargs):

        # print('input_ids.shape: ', input_ids.shape)
        embedding = self.feat_extractor(input_ids, attention_mask)

        if 'dpr' in self.feat_extractor_name.lower():
            pooled = embedding.pooler_output
        else:
            pooled = self.pool(embedding, attention_mask, pool_type='mean')
        # print('embedding.shape: ', embedding.last_hidden_state.shape)
        # last_hidden_states = embedding.last_hidden_state
        # print('last_hidden_states.shape: ', last_hidden_states.shape)
        # pooled = self.pool(last_hidden_states, attention_mask, pool_type='mean')
        # print('pooled.shape: ', pooled.shape)

        if self.normalize:
            pooled = F.normalize(pooled, p=2, dim=1)

        # print(pooled.shape)
        return pooled
    


In [6]:
model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"

#The following is a bigger model and might require slight modification in the code
# follow this link for more details: https://huggingface.co/intfloat/e5-mistral-7b-instruct
# model_ckpt = "intfloat/e5-mistral-7b-instruct"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
embedder = Transformer_embedder(model_ckpt)
embedder = embedder.to(device)

def get_embeddings(text_list):
    encoded_input = embedder.tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    embedder.eval()
    with torch.inference_mode():
        encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
        model_output = embedder(**encoded_input)
    return model_output



In [7]:
question = "Batman"
question_embedding = get_embeddings([question]).cpu().detach().numpy()[0]
question_embedding.shape

(768,)

## Vector Indexing

### Inverted File Indexing

Inverted file indexing is a technique used to index documents in a corpus. It is a data structure that maps terms to the documents that contain them. The inverted file index is a dictionary where the keys are terms and the values are lists of document IDs. The inverted file index is used to quickly find documents that contain a given term.

Mathematically, the inverted file index is defined as follows:

$$
I = \{t_1: [d_1, d_2, \ldots, d_n], t_2: [d_1, d_2, \ldots, d_n], \ldots, t_m: [d_1, d_2, \ldots, d_n]\}
$$

Where $I$ is the inverted file index, $t_i$ is the $i$-th term, and $d_i$ is the $i$-th document ID.





In [26]:
from sklearn.cluster import MiniBatchKMeans


class IVFFlatIndexer:
    def __init__(self, n_clusters: int = 100, n_init: int = 10, max_iter: int = 300):
        self.n_clusters = n_clusters
        self.n_init = n_init
        self.max_iter = max_iter
        self.index = MiniBatchKMeans(
            n_clusters=self.n_clusters,
            n_init=self.n_init,
            max_iter=self.max_iter,
            init_size=3 * self.n_clusters,
        )

    def fit(self, X):
        self.index.fit(X)

    def search(self, X, top_k=5):
        return self.index.predict(X)
    
    def get_cluster_centers(self):
        return self.index.cluster_centers_
    
    def get_cluster_labels(self):
        return self.index.labels_
    

indexer = IVFFlatIndexer(n_clusters=100)
indexer.fit(imdb_embeddings)

print(indexer.search(question_embedding.reshape(1, -1)))

def get_top_k_similar(question_embedding, embeddings, indexer, k=5):
    cluster_id = indexer.search(question_embedding.reshape(1, -1))
    cluster_embeddings = embeddings[indexer.get_cluster_labels() == cluster_id]
    distances = np.dot(cluster_embeddings, question_embedding)
    top_k_indices = np.argsort(distances)[::-1][:k]
    return top_k_indices

top_k_indices = get_top_k_similar(question_embedding, imdb_embeddings, indexer, k=5)
top_k_indices

print("Question: ", question)
print("Top 5 similar movies:")
print("="*100)
for idx in top_k_indices:
    print(imdb_dataset[idx.item()]["Movie Name"])
    print(imdb_dataset[idx.item()]["Plot"])
    print("="*100)

[24]
Question:  Batman
Top 5 similar movies:
Pulp Fiction
The lives of two mob hitmen, a boxer, a gangster and his wife, and a pair of diner bandits intertwine in four tales of violence and redemption.
The Chaos Class
Lazy, uneducated students share a very close bond. They live together in the dormitory, where they plan their latest pranks. When a new headmaster arrives, the students naturally try to overthrow him. A comic war of nitwits follows.
The Marathon Family
The Topalovic family has been in the burial business for generations. When the old (150 yrs old) Pantelija dies, five generations of his heirs start to fight for the inheritance.
The Lord of the Rings: The Fellowship of the Ring
A meek Hobbit from the Shire and eight companions set out on a journey to destroy the powerful One Ring and save Middle-earth from the Dark Lord Sauron.
Goodfellas
The story of Henry Hill and his life in the mafia, covering his relationship with his wife Karen and his mob partners Jimmy Conway and T

### Locality Sensitive Hashing

Local sensitive hashing (LSH) is a technique used to find similar items in a large dataset. LSH is used to reduce the dimensionality of the data and to find similar items in the reduced space. LSH is used in many applications, such as near-duplicate detection, recommendation systems, and clustering.

Mathematically, LSH is defined as follows:

$$
h(x) = \left\{
\begin{array}{ll}
1 & \text{if } x \geq t \\
0 & \text{if } x < t
\end{array}
\right.
$$

Where $h(x)$ is the hash function, $x$ is the input value, and $t$ is the threshold value.

In [39]:
class LSHIndexer:
    def __init__(self, n_bits: int = 8):
        self.n_bits = n_bits

    def fit(self, X):
        self.index = np.packbits(X > 0, axis=1)

    def search(self, X, top_k=5):
        query = np.packbits(X > 0, axis=1)
        distances = np.dot(query, self.index.T)
        top_k_indices = np.argsort(distances)[::-1][:top_k]
        return top_k_indices
    
    def get_index(self):
        return self.index
    

    

indexer = LSHIndexer(n_bits=8)
indexer.fit(imdb_embeddings)

top_k_indices = indexer.search(question_embedding.reshape(1, -1))
top_k_indices = top_k_indices[0][:5]


print("Question: ", question)
print("Top 5 similar movies:")
print("="*100)
for idx in top_k_indices:
    idx = idx.item()
    print(imdb_dataset[idx]["Movie Name"])
    print(imdb_dataset[idx]["Plot"])
    print("="*100)


Question:  Batman
Top 5 similar movies:
The Kids Are All Right
Two children conceived by artificial insemination bring their biological father into their non-traditional family life.
Crimes and Misdemeanors
An ophthalmologist's mistress threatens to reveal their affair to his wife while a married documentary filmmaker is infatuated with another woman.
The Battle of Britain
In 1940, the British Royal Air Force fights a desperate battle to prevent the Luftwaffe from gaining air superiority over the English Channel as a prelude to a possible Axis invasion of the U.K.
Apocalypto
As the Mayan kingdom faces its decline, a young man is taken on a perilous journey to a world ruled by fear and oppression.
Mr. Magorium's Wonder Emporium
The young apprentice of a magical, eccentric toy store owner learns to believe in herself, and in her friends, upon learning some grave news about the future.


### Product Quantization

Product quantization is a technique used to compress high-dimensional vectors into low-dimensional vectors. Product quantization is used to reduce the storage and computation costs of working with high-dimensional vectors. Product quantization is used in many applications, such as image retrieval, text search, and recommendation systems.

Mathematically, product quantization is defined as follows:

$$
Q(x) = \sum_{i=1}^{n} c_i \cdot q_i(x)
$$

Where $Q(x)$ is the quantized vector, $x$ is the input vector, $c_i$ is the $i$-th codebook vector, and $q_i(x)$ is the $i$-th quantization function.

In [44]:
class ProductQuantization:
    def __init__(self, sentence_embeddings, n_subvectors=8):
        """
        Product Quantization Index
        
        Args:
            sentence_embeddings (np.array): Array of sentence embeddings
            n_subvectors (int): Number of subvectors to divide each embedding into
        """
        # Subspace dimensionality
        self.d = sentence_embeddings.shape[1] // n_subvectors
        # Number of subvectors
        self.m = n_subvectors

        # Generate random projection matrix
        self.R = np.random.randn(self.d * self.m, self.d)
        self.R /= np.linalg.norm(self.R, axis=0)

        # Project sentence embeddings onto subspaces
        self.subspace_embeddings = np.dot(sentence_embeddings, self.R)

        # Quantize subspace embeddings
        self.quantized_subspace_embeddings = np.round(self.subspace_embeddings / self.d)

    def search(self, query_embedding, top_k=1):
        """
        Search for the most similar sentences to a query embedding

        Args:
            query_embedding (np.array): Query embedding
            top_k (int): Number of similar sentences to return

        Returns:
            list: List of indices of similar sentences
        """ 
        # Project query embedding onto subspaces
        query_subspace_embeddings = np.dot(query_embedding, self.R)

        # Quantize query subspace embeddings
        quantized_query_subspace_embeddings = np.round(query_subspace_embeddings / self.d)

        # Find nearest neighbors in each subspace
        nearest_neighbors = []
        for i in range(self.m):
            distances = np.linalg.norm(self.quantized_subspace_embeddings[:, i] - quantized_query_subspace_embeddings[i], ord=2)
            nearest_neighbors.append(np.argsort(distances)[:top_k])

        print(nearest_neighbors)
        # Combine nearest neighbors from each subspace
        combined_nearest_neighbors = set()
        for neighbors in nearest_neighbors:
            combined_nearest_neighbors.update(neighbors)

        return list(combined_nearest_neighbors)
    

product_quantizer = ProductQuantization(imdb_embeddings, n_subvectors=8)
top_k_indices = product_quantizer.search(question_embedding, top_k=5)
print(top_k_indices)

print("Question: ", question)
print("Top 5 similar movies:")
print("="*100)
for idx in top_k_indices:
    idx = idx.item()
    print(imdb_dataset[idx]["Movie Name"])
    print(imdb_dataset[idx]["Plot"])
    print("="*100)



[array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0])]
[0]
Question:  Batman
Top 5 similar movies:
The Shawshank Redemption
Over the course of several years, two convicts form a friendship, seeking consolation and, eventually, redemption through basic compassion.


Navigable Small World Graphs

Navigable small world graphs are a technique used to build a graph that can be used to efficiently search for similar items in a large dataset. Navigable small world graphs are used to reduce the search time and space required to find similar items in a large dataset. 

Mathematically, navigable small world graphs are defined as follows:

$$
G = (V, E)
$$

Where $G$ is the navigable small world graph, $V$ is the set of vertices, and $E$ is the set of edges.

The algorithm for building a navigable small world graph is as follows:

1. Initialize the graph with a single vertex.
2. Add a new vertex to the graph.
3. Connect the new vertex to the nearest vertex in the graph.
4. Repeat steps 2 and 3 until all vertices are connected.


In [47]:
!pip install hnswlib

/home/kirekara/anaconda3/lib/python3.11/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hnswlib: filename=hnswlib-0.8.0-cp311-cp311-linux_x86_64.whl size=202917 sha256=9210afb6014dba051395ee1d9874452c5a4bd60c3eac0cc5c7c61393b29b894c
  Stored in directory: /home/kirekara/.cache/pip/wheels/ea/4e/27/39aebca9958719776e36fada290845a7ef10f053ad70e22ceb
Successfully built hnswlib

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [49]:
import hnswlib
class HNSWIndexer:

    def __init__(self, sentence_embeddings, ef=50, M=16):
        """
        HNSW Index
        
        Args:
            sentence_embeddings (np.array): Array of sentence embeddings
            ef (int): Number of neighbors to inspect during search
            M (int): Number of neighbors to keep in graph
        """
        self.ef = ef
        self.M = M
        self.index = hnswlib.Index(space='cosine', dim=sentence_embeddings.shape[1])
        self.index.init_index(max_elements=sentence_embeddings.shape[0], ef_construction=self.ef, M=self.M)
        self.index.add_items(sentence_embeddings)

    def search(self, query_embedding, top_k=1):
        """
        Search for the most similar sentences to a query embedding

        Args:
            query_embedding (np.array): Query embedding
            top_k (int): Number of similar sentences to return

        Returns:
            list: List of indices of similar sentences
        """ 
        self.index.set_ef(self.ef)
        labels, distances = self.index.knn_query(query_embedding, k=top_k)
        return labels

hnsw_indexer = HNSWIndexer(imdb_embeddings)
top_k_indices = hnsw_indexer.search(question_embedding, top_k=5)
print(top_k_indices)

print("Question: ", question)
print("Top 5 similar movies:")
print("="*100)

for idx in top_k_indices[0]:
    idx = idx.item()
    print(imdb_dataset[idx]["Movie Name"])
    print(imdb_dataset[idx]["Plot"])
    print("="*100)



[[1692  242 5220 3285 5994]]
Question:  Batman
Top 5 similar movies:
Batman
The Dark Knight of Gotham City begins his war on crime with his first major enemy being Jack Napier, a criminal who becomes the clownishly homicidal Joker.
Batman Begins
After witnessing his parents' death, Bruce learns the art of fighting to confront injustice. When he returns to Gotham as Batman, he must stop a secret society that intends to destroy the city.
Batman: Gotham by Gaslight
In an alternative Victorian Age Gotham City, Batman begins his war on crime while he investigates a new series of murders by Jack the Ripper.
Batman Returns
While Batman deals with a deformed man calling himself the Penguin wreaking havoc across Gotham with the help of a cruel businessman, a female employee of the latter becomes the Catwoman with her own vendetta.
Batman: The Movie
The Dynamic Duo faces four supervillains who plan to hold the world for ransom with the help of a secret invention that instantly dehydrates people.